# scikit-learn 初級チュートリアル

このチュートリアルでは、Python の機械学習ライブラリ **scikit-learn** の基本を学びます。教師あり学習の基礎から、モデルの評価方法まで習得しましょう。

## 学習内容
1. scikit-learn の概要と基本的な使い方
2. データの前処理
3. 線形回帰
4. ロジスティック回帰（分類）
5. 決定木
6. モデルの評価
7. 交差検証

## 環境設定

In [ ]:
# JupyterLite 環境でのパッケージインストール
import sys
if 'pyodide' in sys.modules:
    import piplite
    await piplite.install(['numpy', 'pandas', 'matplotlib', 'scikit-learn'])

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
print(f'scikit-learn version: {sklearn.__version__}')

# 警告を抑制
import warnings
warnings.filterwarnings('ignore')

# 乱数シード
np.random.seed(42)

---
## 1. scikit-learn の概要と基本的な使い方

### 1.1 scikit-learn の特徴

- **一貫したAPI**: すべてのモデルが同じインターフェースを持つ
  - `fit()`: モデルの学習
  - `predict()`: 予測
  - `score()`: 評価
- **豊富なアルゴリズム**: 回帰、分類、クラスタリング、次元削減など
- **前処理ツール**: スケーリング、エンコーディング、欠損値処理
- **モデル評価**: 交差検証、グリッドサーチなど

### 1.2 サンプルデータセット

In [ ]:
from sklearn.datasets import load_iris, load_wine, load_diabetes

# iris データセット（分類用）
iris = load_iris()
print('=== Iris Dataset ===')
print(f'特徴量: {iris.feature_names}')
print(f'クラス: {iris.target_names}')
print(f'データ形状: {iris.data.shape}')

In [ ]:
# DataFrameに変換
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris['target'] = iris.target
df_iris['species'] = df_iris['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})
print(df_iris.head())

In [ ]:
# diabetes データセット（回帰用）
diabetes = load_diabetes()
print('\n=== Diabetes Dataset ===')
print(f'特徴量: {diabetes.feature_names}')
print(f'データ形状: {diabetes.data.shape}')
print(f'ターゲット形状: {diabetes.target.shape}')

### 1.3 基本的なワークフロー

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 1. データの分割
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f'訓練データ: {X_train.shape}')
print(f'テストデータ: {X_test.shape}')

In [ ]:
# 2. モデルの作成と学習
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

print('モデル学習完了')

In [ ]:
# 3. 予測
y_pred = model.predict(X_test)
print(f'予測結果（先頭10件）: {y_pred[:10]}')
print(f'正解ラベル（先頭10件）: {y_test[:10]}')

In [ ]:
# 4. 評価
accuracy = accuracy_score(y_test, y_pred)
print(f'正解率: {accuracy:.4f}')

# score メソッドでも同じ結果
print(f'score: {model.score(X_test, y_test):.4f}')

---
## 2. データの前処理

### 2.1 特徴量のスケーリング

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# サンプルデータ
X = np.array([[1, 100],
              [2, 200],
              [3, 300],
              [4, 400],
              [5, 500]])

print('元のデータ:')
print(X)

In [ ]:
# 標準化（平均0、標準偏差1）
scaler_standard = StandardScaler()
X_standard = scaler_standard.fit_transform(X)

print('標準化後:')
print(X_standard)
print(f'平均: {X_standard.mean(axis=0)}')
print(f'標準偏差: {X_standard.std(axis=0)}')

In [ ]:
# Min-Max スケーリング（0-1の範囲）
scaler_minmax = MinMaxScaler()
X_minmax = scaler_minmax.fit_transform(X)

print('Min-Max スケーリング後:')
print(X_minmax)
print(f'最小値: {X_minmax.min(axis=0)}')
print(f'最大値: {X_minmax.max(axis=0)}')

### 2.2 カテゴリ変数のエンコーディング

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# サンプルデータ
colors = ['red', 'blue', 'green', 'red', 'blue']

# ラベルエンコーディング
label_encoder = LabelEncoder()
colors_encoded = label_encoder.fit_transform(colors)

print('ラベルエンコーディング:')
print(f'元: {colors}')
print(f'変換後: {colors_encoded}')
print(f'クラス: {label_encoder.classes_}')

In [ ]:
# One-Hot エンコーディング
onehot_encoder = OneHotEncoder(sparse_output=False)
colors_onehot = onehot_encoder.fit_transform(np.array(colors).reshape(-1, 1))

print('One-Hot エンコーディング:')
print(colors_onehot)
print(f'カテゴリ: {onehot_encoder.categories_}')

### 2.3 欠損値の処理

In [ ]:
from sklearn.impute import SimpleImputer

# 欠損値を含むデータ
X_missing = np.array([[1, 2, np.nan],
                      [3, np.nan, 6],
                      [7, 8, 9],
                      [np.nan, 11, 12]])

print('欠損値を含むデータ:')
print(X_missing)

In [ ]:
# 平均値で補完
imputer_mean = SimpleImputer(strategy='mean')
X_imputed_mean = imputer_mean.fit_transform(X_missing)

print('平均値で補完:')
print(X_imputed_mean)

In [ ]:
# 中央値で補完
imputer_median = SimpleImputer(strategy='median')
X_imputed_median = imputer_median.fit_transform(X_missing)

print('中央値で補完:')
print(X_imputed_median)

### 練習問題 2

iris データセットの特徴量を標準化し、訓練データとテストデータに分割してください。

In [ ]:
# 練習問題 2 の解答をここに書いてください


---
## 3. 線形回帰

### 3.1 単回帰

In [ ]:
from sklearn.linear_model import LinearRegression

# サンプルデータの作成
np.random.seed(42)
X = np.random.rand(100, 1) * 10
y = 2 * X.flatten() + 3 + np.random.randn(100) * 2

# 可視化
plt.figure(figsize=(8, 5))
plt.scatter(X, y, alpha=0.7)
plt.xlabel('X')
plt.ylabel('y')
plt.title('Sample Data for Regression')
plt.show()

In [ ]:
# モデルの学習
model = LinearRegression()
model.fit(X, y)

print(f'切片 (intercept): {model.intercept_:.4f}')
print(f'係数 (coefficient): {model.coef_[0]:.4f}')

In [ ]:
# 回帰直線の可視化
y_pred = model.predict(X)

plt.figure(figsize=(8, 5))
plt.scatter(X, y, alpha=0.7, label='Data')
plt.plot(X, y_pred, color='red', linewidth=2, label='Regression Line')
plt.xlabel('X')
plt.ylabel('y')
plt.title(f'y = {model.coef_[0]:.2f}x + {model.intercept_:.2f}')
plt.legend()
plt.show()

### 3.2 重回帰

In [ ]:
# diabetes データセットを使用
X = diabetes.data
y = diabetes.target

# データ分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# モデルの学習
model = LinearRegression()
model.fit(X_train, y_train)

# 係数の確認
print('各特徴量の係数:')
for name, coef in zip(diabetes.feature_names, model.coef_):
    print(f'  {name}: {coef:.2f}')

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# 予測と評価
y_pred = model.predict(X_test)

print(f'MSE: {mean_squared_error(y_test, y_pred):.2f}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}')
print(f'R²: {r2_score(y_test, y_pred):.4f}')

In [ ]:
# 予測値 vs 実測値のプロット
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', linewidth=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted')
plt.show()

---
## 4. ロジスティック回帰（分類）

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# iris データセット（2クラスに限定）
X = iris.data[iris.target != 2]
y = iris.target[iris.target != 2]

# データ分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# モデルの学習
model = LogisticRegression()
model.fit(X_train, y_train)

# 予測
y_pred = model.predict(X_test)

print('=== 分類結果 ===')
print(f'正解率: {accuracy_score(y_test, y_pred):.4f}')

In [ ]:
# 混同行列
cm = confusion_matrix(y_test, y_pred)
print('混同行列:')
print(cm)

# 可視化
plt.figure(figsize=(6, 5))
plt.imshow(cm, cmap='Blues')
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
for i in range(2):
    for j in range(2):
        plt.text(j, i, cm[i, j], ha='center', va='center', fontsize=20)
plt.show()

In [ ]:
# 分類レポート
print('分類レポート:')
print(classification_report(y_test, y_pred, target_names=['setosa', 'versicolor']))

In [ ]:
# 確率の予測
y_prob = model.predict_proba(X_test)
print('予測確率（先頭5件）:')
print(y_prob[:5])

### 4.2 多クラス分類

In [ ]:
# 全クラスを使用
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 多クラス分類
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(f'正解率: {accuracy_score(y_test, y_pred):.4f}')
print('\n分類レポート:')
print(classification_report(y_test, y_pred, target_names=iris.target_names))

---
## 5. 決定木

### 5.1 決定木分類器

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

# データ準備
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# モデルの学習
tree_model = DecisionTreeClassifier(max_depth=3, random_state=42)
tree_model.fit(X_train, y_train)

# 評価
y_pred = tree_model.predict(X_test)
print(f'正解率: {accuracy_score(y_test, y_pred):.4f}')

In [ ]:
# 決定木の可視化
plt.figure(figsize=(15, 10))
plot_tree(tree_model, 
          feature_names=iris.feature_names,
          class_names=iris.target_names,
          filled=True,
          rounded=True)
plt.title('Decision Tree')
plt.show()

In [ ]:
# 特徴量の重要度
importance = tree_model.feature_importances_

plt.figure(figsize=(8, 5))
plt.barh(iris.feature_names, importance)
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.show()

### 5.2 決定木回帰

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# サンプルデータ
np.random.seed(42)
X = np.sort(5 * np.random.rand(100, 1), axis=0)
y = np.sin(X).ravel() + np.random.randn(100) * 0.1

# 異なる深さのモデル
depths = [2, 4, 8]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, depth in zip(axes, depths):
    model = DecisionTreeRegressor(max_depth=depth)
    model.fit(X, y)
    
    X_test = np.linspace(0, 5, 100).reshape(-1, 1)
    y_pred = model.predict(X_test)
    
    ax.scatter(X, y, alpha=0.5, label='Data')
    ax.plot(X_test, y_pred, color='red', linewidth=2, label='Prediction')
    ax.set_title(f'max_depth={depth}')
    ax.legend()

plt.tight_layout()
plt.show()

---
## 6. モデルの評価

### 6.1 分類モデルの評価指標

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, 
    f1_score, roc_auc_score, roc_curve
)

# 2クラス分類の例
X = iris.data[iris.target != 2]
y = iris.target[iris.target != 2]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print('=== 分類評価指標 ===')
print(f'Accuracy:  {accuracy_score(y_test, y_pred):.4f}')
print(f'Precision: {precision_score(y_test, y_pred):.4f}')
print(f'Recall:    {recall_score(y_test, y_pred):.4f}')
print(f'F1 Score:  {f1_score(y_test, y_pred):.4f}')
print(f'AUC:       {roc_auc_score(y_test, y_prob):.4f}')

In [ ]:
# ROC曲線
fpr, tpr, thresholds = roc_curve(y_test, y_prob)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {roc_auc_score(y_test, y_prob):.4f}')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

### 6.2 回帰モデルの評価指標

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# diabetes データセット
X = diabetes.data
y = diabetes.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('=== 回帰評価指標 ===')
print(f'MAE:  {mean_absolute_error(y_test, y_pred):.4f}')
print(f'MSE:  {mean_squared_error(y_test, y_pred):.4f}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}')
print(f'R²:   {r2_score(y_test, y_pred):.4f}')

---
## 7. 交差検証

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

# iris データセット
X = iris.data
y = iris.target

# モデル
model = LogisticRegression(max_iter=200)

# 5分割交差検証
scores = cross_val_score(model, X, y, cv=5)

print('交差検証スコア:')
print(f'各フォールドのスコア: {scores}')
print(f'平均スコア: {scores.mean():.4f}')
print(f'標準偏差: {scores.std():.4f}')

In [ ]:
# KFold を明示的に使用
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, cv=kfold)

print('シャッフル付き交差検証:')
print(f'平均スコア: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})')

In [ ]:
# 異なる評価指標での交差検証
from sklearn.model_selection import cross_validate

scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
results = cross_validate(model, X, y, cv=5, scoring=scoring)

print('複数指標での交差検証:')
for metric in scoring:
    scores = results[f'test_{metric}']
    print(f'{metric}: {scores.mean():.4f} (+/- {scores.std():.4f})')

### 練習問題 7

wine データセットを使って以下を実行してください。

1. 決定木分類器を学習させる（max_depth=5）
2. 5分割交差検証でモデルを評価する
3. 特徴量の重要度を棒グラフで可視化する

In [ ]:
# 練習問題 7 の解答をここに書いてください


---
## まとめ

このチュートリアルで学んだ内容：

| トピック | 主なクラス・関数 |
|---------|---------------|
| 前処理 | `StandardScaler`, `MinMaxScaler`, `LabelEncoder`, `SimpleImputer` |
| 線形回帰 | `LinearRegression` |
| 分類 | `LogisticRegression`, `DecisionTreeClassifier` |
| 評価（分類） | `accuracy_score`, `precision_score`, `recall_score`, `f1_score`, `roc_auc_score` |
| 評価（回帰） | `mean_squared_error`, `mean_absolute_error`, `r2_score` |
| 交差検証 | `cross_val_score`, `cross_validate`, `KFold` |

---
## 練習問題の解答例

In [ ]:
# 練習問題 2 の解答例
print('--- 練習問題 2 ---')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# データ
X = iris.data
y = iris.target

# 訓練・テスト分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 標準化（訓練データでfitしてテストデータに適用）
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f'訓練データ形状: {X_train_scaled.shape}')
print(f'訓練データ平均: {X_train_scaled.mean(axis=0)}')
print(f'訓練データ標準偏差: {X_train_scaled.std(axis=0)}')

In [ ]:
# 練習問題 7 の解答例
print('--- 練習問題 7 ---')

from sklearn.datasets import load_wine
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

# データ読み込み
wine = load_wine()
X = wine.data
y = wine.target

# 1. 決定木分類器
tree_model = DecisionTreeClassifier(max_depth=5, random_state=42)
tree_model.fit(X, y)

# 2. 交差検証
scores = cross_val_score(tree_model, X, y, cv=5)
print(f'交差検証スコア: {scores}')
print(f'平均スコア: {scores.mean():.4f} (+/- {scores.std():.4f})')

# 3. 特徴量重要度の可視化
importance = tree_model.feature_importances_
indices = np.argsort(importance)[::-1]

plt.figure(figsize=(10, 6))
plt.barh([wine.feature_names[i] for i in indices], importance[indices])
plt.xlabel('Importance')
plt.title('Feature Importance (Wine Dataset)')
plt.tight_layout()
plt.show()